In [1]:
import numpy as np
from scipy import sparse
import sys
import os
import time

In [2]:
np.random.seed(0)

## Sparse .npz (compressed) takes the least disk space regardless of density
Uncompressed sparse matrices take more disk space when the matrices are very dense

In [3]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    mat = np.load(f'data/{i}-dense-uncompressed.npy')
    print(i, mat.shape)
    
    density = (mat != 0).sum() / np.prod(mat.shape)
    print(f'density: {density:.3%}')

    sparse_mat = sparse.csc_matrix(mat, copy=True)

    sparse.save_npz(f'data/{i}-sparse-compressed.npz', sparse_mat, compressed=True)
    sparse.save_npz(f'data/{i}-sparse-uncompressed.npz', sparse_mat, compressed=False)
    
    dense_size = os.path.getsize(f'data/{i}-dense-uncompressed.npy') / 1000000
    sparse_size = os.path.getsize(f'data/{i}-sparse-compressed.npz') / 1000000
    sparse_un_size = os.path.getsize(f'data/{i}-sparse-uncompressed.npz') / 1000000
    print(f'dense-uncompressed: {dense_size :.4} MB\nsparse-compressed: {sparse_size :.4g} MB\nsparse-uncompressed: {sparse_un_size :.4} MB\n')

CCpGeAeGaD (1391, 137)
density: 97.807%
dense-uncompressed: 1.525 MB
sparse-compressed: 1.347 MB
sparse-uncompressed: 2.238 MB

DaGpBP (137, 11381)
density: 21.031%
dense-uncompressed: 12.47 MB
sparse-compressed: 2.529 MB
sparse-uncompressed: 3.982 MB

MFpGdCcSE (2884, 5734)
density: 14.093%
dense-uncompressed: 132.3 MB
sparse-compressed: 17.79 MB
sparse-uncompressed: 27.99 MB

GiGpBP (20945, 11381)
density: 4.650%
dense-uncompressed: 1.907e+03 MB
sparse-compressed: 73.81 MB
sparse-uncompressed: 133.1 MB



## Memory-map reduces matrix-vector multiplication time for lower-density matrices

Putting a normal load above a memmap load decreases memmap load time. However, loading memmap first does not speed up normal load times. Note, this only appears to be the case for very dense matrices. For best comparisons, we should always load a memmap matrix first in the sequence.

In [4]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    print(i)
    mat = np.load(f'data/{i}-dense-uncompressed.npy')
    
    density = 100 * (mat != 0).sum() / np.prod(mat.shape)
    print(f'density: {density :.3} %')

    # Create a vector to multiply
    vector_size = mat.shape[0]
    vec = np.zeros((1, vector_size))
    # 4 search nodes
    indices = np.random.randint(0, high=vector_size, size=4)
    vec[0, indices] = 1
    del mat
    
    t1 = time.time()
    matrix = np.load(f'data/{i}-dense-uncompressed.npy', mmap_mode='r')
    output = vec @ matrix
    t2 = time.time()
    time_2 = t2 - t1
    del matrix
    del output
    
    t1 = time.time()
    matrix = np.load(f'data/{i}-dense-uncompressed.npy', mmap_mode=None)
    output = vec @ matrix
    t2 = time.time()
    time_1 = t2 - t1
    del matrix
    del output
    
    # Second memory-map load time for comparison
    t1 = time.time()
    matrix = np.load(f'data/{i}-dense-uncompressed.npy', mmap_mode='r')
    output = vec @ matrix
    t2 = time.time()
    time_3 = t2 - t1
    del matrix
    del output
      
    print(f'Normal: {1000 * time_1 :.3} ms\nMMAP: {1000 * time_2 :.3} ms\nMMAP2: {1000 * time_3 :.3} ms\n')

CCpGeAeGaD
density: 97.8 %
Normal: 0.383 ms
MMAP: 0.796 ms
MMAP2: 0.302 ms

DaGpBP
density: 21.0 %
Normal: 2.08 ms
MMAP: 0.9 ms
MMAP2: 0.93 ms

MFpGdCcSE
density: 14.1 %
Normal: 48.7 ms
MMAP: 5.86 ms
MMAP2: 6.14 ms

GiGpBP
density: 4.65 %
Normal: 5.4e+02 ms
MMAP: 69.7 ms
MMAP2: 68.0 ms



## Sparse (uncompressed/compressed) vs Dense

Uncompressed sparse matrices load and multiply about as quickly as dense matrices. The order of matrix reads does not change for sparse reads from npz files. Compressed sparse matrices take an order of magnitude more time to load and multiply.

In [5]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    print(i)
    mat = np.load(f'data/{i}-dense-uncompressed.npy')
    
    density = 100 * (mat != 0).sum() / np.prod(mat.shape)
    print(f'density: {density :.3} %')

    # Create a vector to multiply
    vector_size = mat.shape[0]
    vec = np.zeros((1, vector_size))
    # 4 search nodes
    indices = np.random.randint(0, high=vector_size, size=4)
    vec[0, indices] = 1
    del mat
    
    t1 = time.time()
    matrix = np.load(f'data/{i}-dense-uncompressed.npy')
    output = vec @ matrix
    t2 = time.time()
    time_1 = t2 - t1
    del matrix
    del output
    
    t1 = time.time()
    matrix = sparse.load_npz(f'data/{i}-sparse-compressed.npz')
    output = vec @ matrix
    t2 = time.time()
    time_2 = t2 - t1
    del matrix
    del output
    
    t1 = time.time()
    matrix = sparse.load_npz(f'data/{i}-sparse-uncompressed.npz')
    output = vec @ matrix
    t2 = time.time()
    time_3 = t2 - t1
    del matrix
    del output
    
    print(f'Dense No Memmap: {1000 * time_1 :.3} ms\nCompressed Sparse: {1000 * time_2 :.3} ms\nUncompressed Sparse: {1000 * time_3 :.3} ms\n')

CCpGeAeGaD
density: 97.8 %
Dense No Memmap: 0.579 ms
Compressed Sparse: 8.56 ms
Uncompressed Sparse: 3.22 ms

DaGpBP
density: 21.0 %
Dense No Memmap: 2.15 ms
Compressed Sparse: 14.9 ms
Uncompressed Sparse: 4.72 ms

MFpGdCcSE
density: 14.1 %
Dense No Memmap: 36.9 ms
Compressed Sparse: 1.01e+02 ms
Uncompressed Sparse: 25.1 ms

GiGpBP
density: 4.65 %
Dense No Memmap: 5.31e+02 ms
Compressed Sparse: 5e+02 ms
Uncompressed Sparse: 1.4e+02 ms



## Subset matrix memmap based on nonzero rows of the search vector

In [6]:
for i in ['CCpGeAeGaD', 'DaGpBP', 'MFpGdCcSE', 'GiGpBP']:
    print(i)
    mat = np.load(f'data/{i}-dense-uncompressed.npy')
    print(mat.shape)
    
    # Create a vector to multiply
    vector_size = mat.shape[0]
    vec4 = np.zeros((1, vector_size)).flatten()
    vec10 = np.zeros((1, vector_size)).flatten()
    
    # 4 search nodes
    indices4 = np.random.randint(0, high=vector_size, size=4)
    indices10 = np.random.randint(0, high=vector_size, size=10)
    vec4[indices4] = 1
    vec10[indices10] = 1
    del mat
    print_str = ''
    
    # No memmap
    t1 = time.time()
    matrix = np.load(f'data/{i}-dense-uncompressed.npy', mmap_mode=None)
    output = vec4 @ matrix
    t2 = time.time()
    time_1 = t2 - t1
    del matrix
    del output
    print_str += f'no memmap4: {1000 * time_1 :.3} ms\n'
    
    t1 = time.time()
    matrix = np.load(f'data/{i}-dense-uncompressed.npy', mmap_mode=None)
    output = vec10 @ matrix
    t2 = time.time()
    time_2 = t2 - t1
    del matrix
    del output
    print_str += f'no memmap10: {1000 * time_2 :.3} ms\n'
        
    # Simple memmap
    t1 = time.time()
    matrix = np.load(f'data/{i}-dense-uncompressed.npy', mmap_mode='r')
    output = vec4 @ matrix
    t2 = time.time()
    time_3 = t2 - t1
    del matrix
    del output
    print_str += f'memmap4: {time_3 / time_1 :.3} times no-memmap\n'
    
    t1 = time.time()
    matrix = np.load(f'data/{i}-dense-uncompressed.npy', mmap_mode='r')
    output = vec10 @ matrix
    t2 = time.time()
    time_4 = t2 - t1
    del matrix
    del output
    print_str += f'memmap10: {time_4 / time_2 :.3} times no-memmap\n'
          
    # create new vector of ones
    t1 = time.time()
    matrix = np.load(f'data/{i}-dense-uncompressed.npy', mmap_mode='r')
    output = np.ones(4) @ matrix[indices4]
    t2 = time.time()
    time_5 = t2 - t1
    del matrix
    del output
    print_str += f'subset 4: {time_5 / time_1 :.3} times no-memmap\n'
    
    t1 = time.time()
    matrix = np.load(f'data/{i}-dense-uncompressed.npy', mmap_mode='r')
    output = np.ones(10) @ matrix[indices10]
    t2 = time.time()
    time_6 = t2 - t1
    del matrix
    del output
    print_str += f'subset 10: {time_6 / time_2 :.3} times no-memmap\n'
        
    print(print_str)

CCpGeAeGaD
(1391, 137)
no memmap4: 0.427 ms
no memmap10: 0.339 ms
memmap4: 0.732 times no-memmap
memmap10: 0.855 times no-memmap
subset 4: 0.574 times no-memmap
subset 10: 0.727 times no-memmap

DaGpBP
(137, 11381)
no memmap4: 2.19 ms
no memmap10: 2.15 ms
memmap4: 0.419 times no-memmap
memmap10: 0.353 times no-memmap
subset 4: 0.142 times no-memmap
subset 10: 0.167 times no-memmap

MFpGdCcSE
(2884, 5734)
no memmap4: 36.6 ms
no memmap10: 38.7 ms
memmap4: 0.153 times no-memmap
memmap10: 0.144 times no-memmap
subset 4: 0.0109 times no-memmap
subset 10: 0.00823 times no-memmap

GiGpBP
(20945, 11381)
no memmap4: 5.31e+02 ms
no memmap10: 5.3e+02 ms
memmap4: 0.133 times no-memmap
memmap10: 0.127 times no-memmap
subset 4: 0.06 times no-memmap
subset 10: 0.0746 times no-memmap

